In [ ]:
try:
    import piplite
    await piplite.install(["ipywidgets==8.1.7", "ipyleaflet==0.20.0", "emfs:here_search_demo-0.12.0-py3-none-any.whl"], keep_going=True)
    api_key = "<YOUR HERE API KEY>"
except ImportError:
    api_key = None

# Input widgets

To interact with the [HERE search & geocoding API](https://developer.here.com/documentation/geocoding-search-api/api-reference-swagger.html) services `here_search` provides a few widgets based on [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/index.html):
- `SubmittableTextBox` and `TermsButtons` to formulate textual search intents for Autosuggest and Discover
- `PlaceTaxonomyButtons` to formulate custom place classification shortcuts for Browse
- `OneBoxSimple` to associate search intents to search requests

Those widgets are using a queue to exchange objects of `SearchIntent` class derivatives.

### SubmittableTextBox

`SubmittableTextBox` is a special `Text` widget supporting key strokes events and text submissions.<br/>
It uses a queue to post `SearchIntent` objects, either `TransientTextIntent` for each keystroke or `FornulatedTextIntent` when the transient text is submitted. Search intent is materialized in these objects through their `materialization` attribute, set to a string for these two later classes.

In the snippet below, key strokes and text submissions are displayed in an `Output` widget.

In [ ]:
from here_search.demo.widgets.input import SubmittableTextBox
from here_search.demo.widgets.util import Output
from here_search.demo.entity.intent import FormulatedTextIntent
from asyncio import ensure_future, Queue

box = SubmittableTextBox(Queue())
out_keys, out_text = Output(height=30), Output(height=30)

async def print_text():
    while True:
        intent = await box.queue.get()
        out = out_text if isinstance(intent, FormulatedTextIntent) else out_keys
        with out:
            out.replace(intent.materialization)

t1 = ensure_future(print_text())
display(box, out_keys, out_text)

### TermsButtons

`TermsButtons` objects are taking a `SubmittableTextBox` instance to feed it with predefined values when the buttons are selected.<br/>
Per default, the complete form value is overwritten by the button text. 

The constructor `index` parameter allows to specify wich token is overwritten:

In [ ]:
from here_search.demo.widgets.input import TermsButtons

box2 = SubmittableTextBox(Queue())
out_keys2 = Output(height=30)

async def print_text2():
    while True:
        intent = await box2.queue.get()
        with out_keys2:
            out_keys2.replace(intent.materialization)
            
terms = TermsButtons(box2, 
                     values=["text 1", "text 2"], 
                     index=-1)

t2 = ensure_future(print_text2())
display(box2, terms, out_keys2)

### PlaceTaxonomyButtons

`PlaceTaxonomyButtons` is a widget associating a list of buttons to a place classification.<br/>
It uses a queue to post a `PlaceTaxonomyIntent` object materialized with the place taxonomy item when a button is selected.

The example below uses the `PlaceTaxonomyExample` object. Each buttons selection displays the related taxonomy item in an `Output` widget.

(fontawesome 5.12.0 [download link](https://use.fontawesome.com/releases/v5.12.0/fontawesome-free-5.12.0-web.zip))

In [ ]:
from here_search.demo.entity.place import PlaceTaxonomyExample
from here_search.demo.widgets.input import PlaceTaxonomyButtons

buttons = PlaceTaxonomyButtons(Queue(), PlaceTaxonomyExample.taxonomy, PlaceTaxonomyExample.icons)
out_onto = Output(height=30)

async def print_taxonomy():
    while True:
        with out_onto:
            intent = await buttons.queue.get()
            out_onto.replace(str(intent.materialization))
            
t4 = ensure_future(print_taxonomy())
display(buttons, out_onto)

## ResponseMap

Search end-users fire requests (text queries, or ontology shortcuts) from a particular location context, materialized in the API through the search center lat/lon. It is convenient to capture a search center thrioygh the use of a draggable map.

The `ResponseMap` class uses [`ipywidgets`](https://pypi.org/project/ipywidgets) `Map` class dragging and zooming capabilities to capture the center of a HERE map and sends it to an observer function. This class expects a `position_handler` constructor parameter respecting the traitlets event handler [signature](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Registering-callbacks-to-trait-changes-in-the-kernel). In the snippet below, the map position is only displayed in an `Output` widget, each time the user interacts with the map:

In [ ]:
from here_search.demo.widgets.output import ResponseMap
from here_search.demo.api import API

api = API(api_key=api_key)
out_map = Output(height=30)

def position_handler(latitude, longitude):
    with out_map:
        out_map.replace(f'{latitude},{longitude}')
    
rmap = ResponseMap(api_key=api.api_key, 
                   center=(52.51604, 13.37691),
                   layout = {'height': '200px'},
                   position_handler=position_handler)

display(rmap, out_map)

## OneBoxSimple

`OneBoxSimple` class brings the previously described widgets together to offer both a location aware one-box and a taxonomy search experience.<br/>
For this, a asyncio task is continusouly waiting for search events, and dispatch them to the dedicated search request and response handlers:
- `OneBoxSimple` hosts an instance of `API`, a `search_center` tuple attribute providing a location context and a `language` preference attribute initialized to english.
- `TransientTextIntent` and `FormulatedTextIntent` objects materialization is sent to `API.autosuggest()` and `API.discover()` respectively.
- `PlaceTaxonomyIntent` materialization is sent to `API.browse()`.
- Autosuggest responses are sent to `handle_suggestion_list()`
- Discover and Browse responses are sent `handle_result_list()`

In the snippet below, `App` overwrites the response handlers to display them to a `ResponseMap` and an `Output` widget. The `search_center` attribute is bound to the map center:

In [ ]:
from here_search.demo.base import OneBoxSimple
from json import dumps

queue4 = Queue()
text_box = SubmittableTextBox(queue4)
terms_buttons = TermsButtons(text_box, index=-1)
taxonomy_buttons = PlaceTaxonomyButtons(queue4, PlaceTaxonomyExample.taxonomy, PlaceTaxonomyExample.icons)
rmap = ResponseMap(api_key=api.api_key, center=(52.51604, 13.37691), queue=queue4, layout = {'height': '200px'})
out_simple = Output(height=160)

class App(OneBoxSimple):
    
    def __init__(self, queue):
        super().__init__(api=api, queue=queue)
        def handler(latitude, longitude):
            self.search_center = latitude, longitude
        rmap.set_position_handler(handler)
    
    def handle_suggestion_list(self, resp):
        with out_simple:
            out_simple.replace(dumps(resp.data, indent=2))
        terms_buttons.set(resp.terms)
        
    def handle_result_list(self, resp):
        with out_simple:
            out_simple.replace(dumps(resp.data, indent=2))
        text_box.text_w.value = ''
        terms_buttons.set([])
        rmap.display(resp)

app = App(queue4).run()
display(taxonomy_buttons, text_box, terms_buttons, rmap, out_simple)

<!--
from urllib.parse import quote, unquote
svg = """<svg width="40" height="48" viewBox="0 0 40 48" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="triangle" d="M11,36.8l-5.5,5.5L0,36.8H11z" fill="#48DAD0"></path><path class="HERE" d="M19.1,24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9,1.7-0.5,2.7,0.5L19.1,24.2z M30.9,9.2c0.9-0.9,1.7-0.5,2.7,0.5
    L31.3,12C30.1,10.7,30.2,9.9,30.9,9.2z M38,11.6c-1.1,1.6-2.9,4.2-4.9,2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4
	c-1.6,1.6-2,3.4-1.5,5.1l-1.6-1.8c-0.5,0.3-2.4,1.9-0.9,4.5l-1.8-1.5l-2.4,2.4l3.2,3.2c-2.5-1.9-5.1-1.8-7.2,0.4
	c-2.3,2.3-2.1,5-0.4,7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9,0.9-1.5,2.2-1.3,3.1L4,24.6l-2.6,2.6l9.6,9.6h5.2l-3.5-3.5
	c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8,1.8-0.3,3.6,1.4l3.4,3.4l2.6-2.6l-3.2-3.2c2.3,1.8,5.1,1.9,7.7-0.6l0,0c1.5-1.4,2-2.8,2-2.8
	l-1.9-1.3c-1.1,1.6-2.9,4.2-4.9,2.3l5-5l3.1,3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5,0-4.1c0.4,0.7,0.9,1.5,1.5,2.1
	c2.5,2.5,5.7,3,8.6,0.2l0,0c1.5-1.4,2-2.8,2-2.8S38,11.6,38,11.6z" fill="#000000"></path></svg>
"""
print(f"""![HERE](data:image/svg+xml,{quote(svg)})
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>""")
-->
![HERE](data:image/svg+xml,%3Csvg%20width%3D%2240%22%20height%3D%2248%22%20viewBox%3D%220%200%2040%2048%22%20fill%3D%22none%22%20xmlns%3D%22http%3A//www.w3.org/2000/svg%22%3E%3Cpath%20class%3D%22triangle%22%20d%3D%22M11%2C36.8l-5.5%2C5.5L0%2C36.8H11z%22%20fill%3D%22%2348DAD0%22%3E%3C/path%3E%3Cpath%20class%3D%22HERE%22%20d%3D%22M19.1%2C24.2c-1.2-1.4-1.1-2.1-0.4-2.8c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5L19.1%2C24.2z%20M30.9%2C9.2c0.9-0.9%2C1.7-0.5%2C2.7%2C0.5%0A%20%20%20%20L31.3%2C12C30.1%2C10.7%2C30.2%2C9.9%2C30.9%2C9.2z%20M38%2C11.6c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.2l5-5c-0.4-0.5-0.8-0.9-1-1.1c-2.7-2.7-5.7-2.7-8-0.4%0A%09c-1.6%2C1.6-2%2C3.4-1.5%2C5.1l-1.6-1.8c-0.5%2C0.3-2.4%2C1.9-0.9%2C4.5l-1.8-1.5l-2.4%2C2.4l3.2%2C3.2c-2.5-1.9-5.1-1.8-7.2%2C0.4%0A%09c-2.3%2C2.3-2.1%2C5-0.4%2C7.3l-0.3-0.3c-2.3-2.3-4.7-1.5-5.9-0.3c-0.9%2C0.9-1.5%2C2.2-1.3%2C3.1L4%2C24.6l-2.6%2C2.6l9.6%2C9.6h5.2l-3.5-3.5%0A%09c-1.8-1.8-1.8-2.8-1-3.7c0.8-0.8%2C1.8-0.3%2C3.6%2C1.4l3.4%2C3.4l2.6-2.6l-3.2-3.2c2.3%2C1.8%2C5.1%2C1.9%2C7.7-0.6l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8%0A%09l-1.9-1.3c-1.1%2C1.6-2.9%2C4.2-4.9%2C2.3l5-5l3.1%2C3.1l2.7-2.7l-3.9-3.9c-1.8-1.8-0.7-3.5%2C0-4.1c0.4%2C0.7%2C0.9%2C1.5%2C1.5%2C2.1%0A%09c2.5%2C2.5%2C5.7%2C3%2C8.6%2C0.2l0%2C0c1.5-1.4%2C2-2.8%2C2-2.8S38%2C11.6%2C38%2C11.6z%22%20fill%3D%22%23000000%22%3E%3C/path%3E%3C/svg%3E%0A)
<span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2025 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, 
storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential 
information which may not be disclosed to others without the prior written consent of HERE.</sub></span>